# Simple Filtering

## Summary
In this notebook, we'll be covering:
- [Filtering using mathematical operators](#Filtering-Using-Mathematical-Operators)
- [Filtering using isin()](#Filtering-Using-isin())
- [Chaining filters](#Chaining-Filters)

In this section we will discuss filtering data. The All of Us Researcher Workbench will do *some* filtering for you, but undoubtedly, you will need to perform additional filtering conduct your analysis. Filtering can become complex, so this section is split into two notebooks. This one covers simple filters, while the next one covers creating new data labels that we can then use to further subset our data.

### Filtering Using Mathematical Operators
To get started with filtering, we first need a dataset to use for demonstration. The code below will make a dataframe for us. In fact, it will be the same dataframe as the previous notebook, but without random blank cells.

It is not important to understand all of the code in the cell below. The important thing to know is that we are generating fake data to use for demonstration. Our fake dataset will have the following columns:

* ID
* Measurement Device
* Heart Rate Max
* Heart Rate Min
* Heart Rate Avg
* Duration of exercise (min)
* Exercise Type

In [1]:
import pandas as pd
import random

workout_dict = {'ID': [], 'Measurement Device': [], 'Heart Rate Max': [], 'Heart Rate Min': [], 'Heart Rate Avg': [],
              'Duration of exercise (min)': [], 'Exercise Type': []}
used_ids = []

for x in range(0, 500):
    id = random.randint(100000000, 999999999)
    while id in used_ids:
        id = random.randint(100000000, 999999999)
    used_ids.append(id)
    device = random.choice(['Skykandal', 'B-Wolf'])
    mu = random.randint(65, 85)
    min_rate = int(random.gauss(mu, 10))
    max_rate = int(random.gauss(mu + 55, 25))
    while max_rate <= min_rate:
        max_rate = int(random.gauss(mu + 55, 25))
    avg = random.gauss((max_rate + min_rate) / 2, (max_rate - min_rate) / 5)
    duration = random.randint(10, 90)
    exercise = random.choice(['Running', 'Running', 'Running', 'Bicycling', 'Swimming', 'Swimming',
                              'Weight training'])
    row = [device, min_rate, max_rate, avg, duration, exercise]
    workout_dict['ID'].append(id)
    workout_dict['Measurement Device'].append(row[0])
    workout_dict['Heart Rate Min'].append(row[1])
    workout_dict['Heart Rate Max'].append(row[2])
    workout_dict['Heart Rate Avg'].append(row[3])
    workout_dict['Duration of exercise (min)'].append(row[4])
    workout_dict['Exercise Type'].append(row[5])

df = pd.DataFrame(workout_dict)
df.head(10)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,256993349,B-Wolf,171,98,138.868338,77,Swimming
1,107312079,B-Wolf,97,71,80.514031,70,Swimming
2,497063989,Skykandal,118,79,98.804297,75,Running
3,832893057,Skykandal,99,65,74.610740,90,Weight training
4,157561470,Skykandal,157,76,146.046041,47,Swimming
5,346456009,Skykandal,99,68,84.761159,76,Running
6,114287779,B-Wolf,92,66,81.497908,43,Swimming
7,992102571,B-Wolf,150,71,98.688013,72,Weight training
8,198499198,Skykandal,145,101,126.178451,30,Running
9,894612807,B-Wolf,112,85,102.281878,31,Weight training


Let's start with a simple filter: we will select everyone whose **minimum heart rate** is greater than 69.

In [2]:
df[df['Heart Rate Min'] > 69]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,256993349,B-Wolf,171,98,138.868338,77,Swimming
1,107312079,B-Wolf,97,71,80.514031,70,Swimming
2,497063989,Skykandal,118,79,98.804297,75,Running
4,157561470,Skykandal,157,76,146.046041,47,Swimming
7,992102571,B-Wolf,150,71,98.688013,72,Weight training
...,...,...,...,...,...,...,...
495,821656493,Skykandal,142,82,87.489770,22,Weight training
496,193488022,Skykandal,145,72,115.574089,86,Running
497,162490491,Skykandal,118,85,87.892640,40,Weight training
498,994465628,Skykandal,120,89,112.756578,66,Running


How does this work? Well, we start with `df[]` which is a common way to get something out of dataframe `df` using keywords. Except, instead of passing something like a column name, we passed an expression into the brackets. The expression was `df['Heart Rate Min'] > 69'`. In other words, from df give me every row where Heart Rate Min (in df) is greater than 69.

Now, let's get one of the most annoying features of pandas out of the way. We're going to store our filtered data in df_filtered and then, for no good reason, change a column.

In [3]:
df_filtered = df[df['Heart Rate Min'] > 69]
df_filtered['Heart Rate Avg'] = 200
df_filtered.head()

/var/folders/gb/52m04vs53ln6pc3397t447th0000gq/T/ipykernel_29919/3099549399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Heart Rate Avg'] = 200


,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,256993349,B-Wolf,171,98,200,77,Swimming
1,107312079,B-Wolf,97,71,200,70,Swimming
2,497063989,Skykandal,118,79,200,75,Running
4,157561470,Skykandal,157,76,200,47,Swimming
7,992102571,B-Wolf,150,71,200,72,Weight training


Above you should have gotten a `SettingWithCopyWarning`. You might have assumed that when you assigned `df[df['Age'] > 29]` to `df_filtered` you created an entirely new dataframe, named `df_filtered`. You didn't. You created a "view" of the old dataframe (`df`) that only saw the parts of `df` that you wanted, but wasn't actually a new dataframe. This means that attempting to set a value in `df_filtered` may also alter values in `df`. And so we get a warning.

When you get a view versus when you get a new pandas object is fairly complicated. However, there is a simple fix: `copy` can turn a view into a copy. Copy is a dataframe method, and takes no arguments.

Below, the code has added `copy()` and what previously gave us a warning now won't. To be clear, the main difference is that now `df_filtered` is a *new dataframe* instead of a view of the original dataframe. 

In [4]:
df_filtered = df[df['Heart Rate Min'] > 69].copy()
df_filtered['Heart Rate Avg'] = 200
df_filtered.head()

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,256993349,B-Wolf,171,98,200,77,Swimming
1,107312079,B-Wolf,97,71,200,70,Swimming
2,497063989,Skykandal,118,79,200,75,Running
4,157561470,Skykandal,157,76,200,47,Swimming
7,992102571,B-Wolf,150,71,200,72,Weight training


Filtering expressions can include a number of comparison operators, such as: >, <, <=, >=, ==, and !=. Remember that in Python = means "set the thing on the left side of the = equal to the thing on the right side of the =", whereas == asks if they are equal. != means "not equal".

These expressions are fairly straightforward when we are dealing with numbers. 

#### Below, write some code that filters `df` so that only individuals with a maximum heart rate of 110 or under remain.

In [5]:
# your code goes here


You can also use some of these operators, like == and != to filter on text columns. Below, we'll just select the individuals who went running.

In [6]:
df[df['Exercise Type'] == 'Running']

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
2,497063989,Skykandal,118,79,98.804297,75,Running
5,346456009,Skykandal,99,68,84.761159,76,Running
8,198499198,Skykandal,145,101,126.178451,30,Running
11,328006648,B-Wolf,150,81,95.079276,35,Running
12,147199923,B-Wolf,104,94,97.504354,66,Running
...,...,...,...,...,...,...,...
488,143314360,Skykandal,149,88,106.167148,33,Running
490,712272568,B-Wolf,128,45,112.905751,88,Running
493,839889240,B-Wolf,175,68,93.909092,27,Running
496,193488022,Skykandal,145,72,115.574089,86,Running


We could also *exclud*e people who went running using `~`. This means "not", and it's generally good practice (i.e., it will sometimes break if you don't do this) to wrap the expression you are negating in parentheses, as below.

In [7]:
df[~(df['Exercise Type'] == 'Running')]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,256993349,B-Wolf,171,98,138.868338,77,Swimming
1,107312079,B-Wolf,97,71,80.514031,70,Swimming
3,832893057,Skykandal,99,65,74.610740,90,Weight training
4,157561470,Skykandal,157,76,146.046041,47,Swimming
6,114287779,B-Wolf,92,66,81.497908,43,Swimming
...,...,...,...,...,...,...,...
492,643276129,Skykandal,121,61,86.893537,76,Weight training
494,403938983,Skykandal,168,82,160.210336,82,Bicycling
495,821656493,Skykandal,142,82,87.489770,22,Weight training
497,162490491,Skykandal,118,85,87.892640,40,Weight training


What happens if we use a > or < symbol?

In [8]:
df[df['Exercise Type'] < 'Running']

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
29,616103286,Skykandal,156,74,126.509807,82,Bicycling
32,431411408,Skykandal,98,78,86.424694,44,Bicycling
45,704129462,Skykandal,155,69,113.384224,71,Bicycling
46,449575189,Skykandal,182,84,125.069906,63,Bicycling
50,780828274,B-Wolf,170,96,126.635341,81,Bicycling
60,673182491,Skykandal,108,93,100.778073,85,Bicycling
63,413352614,B-Wolf,146,77,101.320412,33,Bicycling
65,814569321,B-Wolf,145,78,97.006704,57,Bicycling
67,841714588,Skykandal,141,67,89.375990,54,Bicycling
68,515674573,B-Wolf,147,80,94.452482,29,Bicycling


`< 'Running'` gives us only bicyclists, because > and < on text check alphabetization. < 'Running' gave us every label that is, when alphabetized, earlier than "Running". This behavior is rarely useful. It's generally best practice to avoid '<' and '>' with string (i.e., text) columns. 

#### Below, write code that returns only people using the B-Wolf measuring device. Do this two different ways. To make it easier to see that each way worked there are separate cells for each way.

In [ ]:
# your code goes here for the first way


In [ ]:
# the second way goes here


What if we wanted to filter based on the relationship between two columns? We can pass more complex expressions to the filter as well. The filter below will return individuals whose maximum heart rate is within 20 beats of their minimum heart rate.

In [9]:
df[(df['Heart Rate Max'] - df['Heart Rate Min']) <= 20]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
12,147199923,B-Wolf,104,94,97.504354,66,Running
30,507671775,Skykandal,72,63,68.070474,70,Running
32,431411408,Skykandal,98,78,86.424694,44,Bicycling
33,974402758,B-Wolf,97,82,90.259986,43,Running
44,487568600,Skykandal,92,74,83.457171,78,Swimming
56,178202287,Skykandal,89,77,84.248213,82,Swimming
60,673182491,Skykandal,108,93,100.778073,85,Bicycling
75,254302084,Skykandal,94,80,87.367578,35,Swimming
77,450620626,B-Wolf,78,69,71.804605,33,Running
84,377336207,B-Wolf,85,70,75.454617,17,Bicycling


#### Below, write code that returns everyone whose average heart rate was less than the number you would get by averaging the maximum and minimum heart rates. It's easy to miss parentheses and brackets when writing something this long, so I suggest that you write in your parentheses/brackets first, and then fill them in.

In [10]:
# your code goes here


### Filtering Using isin()
The filtering tools above work fine for text columns where we want only one value, but what if we want more than one value? The `isin` method is our answer.

In [ ]:
df[df['Exercise Type'].isin(['Running', 'Swimming'])]

The `isin` method is very similar to the basic Python `in`. It simply checks to see if something is in an iterable that you provide. In this case we simply passed the list `['Running', 'Swimming']` to `isin` and got back every row where the value in the Condition column was in the list.

If we had a lot of categories and we wanted to get most, but not all, of them we could in theory use `isin` and write a long list of categories to be included. However, it would be nicer to exclude the small list. Thankfully, ~ works for `isin` just like it did for mathematical expressions.

Below, we write an expression to return all the rows where the Exercise Type is **not** in the list `['Running', 'Swimming']`.

In [11]:
df[~df['Exercise Type'].isin(['Running', 'Swimming'])]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
3,832893057,Skykandal,99,65,74.610740,90,Weight training
7,992102571,B-Wolf,150,71,98.688013,72,Weight training
9,894612807,B-Wolf,112,85,102.281878,31,Weight training
16,153570473,B-Wolf,169,88,136.191140,35,Weight training
18,798478810,B-Wolf,124,61,74.552100,33,Weight training
...,...,...,...,...,...,...,...
489,798061195,B-Wolf,119,75,95.709232,46,Bicycling
492,643276129,Skykandal,121,61,86.893537,76,Weight training
494,403938983,Skykandal,168,82,160.210336,82,Bicycling
495,821656493,Skykandal,142,82,87.489770,22,Weight training


While you might be tempted to try and write `df.~isin` Python interprets that as nonsense. In reality, `df.isin` is creating a dataframe full of `True` and `False`, and we're asking for that frame to be flipped, so every `True` is a `False` and vice-versa.

#### For a bit of quick practice, make a filter that returns everyone who wasn't bicycling or running.

In [12]:
# your code goes here


### Chaining Filters
Now, we can combine filters by chaining filters together in the following manner:

In [13]:
temp_df = df[df['Heart Rate Min'] > 80]
temp_df[temp_df['Measurement Device'] == 'Skykandal']

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
8,198499198,Skykandal,145,101,126.178451,30,Running
10,189524068,Skykandal,109,83,90.497413,29,Swimming
15,441729444,Skykandal,183,87,88.984691,79,Running
23,128204451,Skykandal,162,84,106.856019,35,Swimming
46,449575189,Skykandal,182,84,125.069906,63,Bicycling
...,...,...,...,...,...,...,...
491,939321705,Skykandal,105,92,99.900711,29,Swimming
494,403938983,Skykandal,168,82,160.210336,82,Bicycling
495,821656493,Skykandal,142,82,87.489770,22,Weight training
497,162490491,Skykandal,118,85,87.892640,40,Weight training


However, this is ugly and requires the creation of potentially a large number of temporary dataframes that exist only to hold intermediate steps. We could do the same thing a different way: write each expression in parentheses and then connect them together.

Below is the same result as above but in one line.

In [ ]:
df[(df['Heart Rate Min'] > 80) & (df['Measurement Device'] == 'Skykandal')]

All we've done here is connect two expressions we already understand with &. Python evaluates each expression (`df['Heart Rate Min'] > 80` and `df['Measurement Device'] == 'Skykandal'`) separately. If they both evaluate to True then the whole statement evaluates to True.

#### Write some code below that filters for all people who were swimming and spent more than 60 minutes doing so.

In [ ]:
# your code goes here


There are other ways to connect expressions. `&` handles "and", meaning both parts have to be true. `|` (which you can generally type as Shift+\) means "or", meaning at least one of the parts has to be true.

This is how `&` and `|` interpret logical expressions:
* True & True = True
* False & True = False
* False | True = True
* True | True = True

Between `&`, `|`, and `~` you can build some fairly complex expressions, especially since you can connect blocks inside parentheses with other blocks.

The code block below filters for everyone with an average heart rate of 120-130 or 140-150. Visually, the line is broken at the | so you can more easily see the two halves.

In [14]:
df[((df['Heart Rate Avg']>= 120) & (df['Heart Rate Avg'] <= 130)) | 
   ((df['Heart Rate Avg'] >= 140) & (df['Heart Rate Avg'] <= 150))]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
4,157561470,Skykandal,157,76,146.046041,47,Swimming
8,198499198,Skykandal,145,101,126.178451,30,Running
14,760138653,B-Wolf,140,82,121.886573,22,Running
29,616103286,Skykandal,156,74,126.509807,82,Bicycling
36,512580058,B-Wolf,201,86,147.932365,28,Running
...,...,...,...,...,...,...,...
448,924174572,Skykandal,190,60,142.296311,69,Swimming
449,508684249,Skykandal,151,82,141.705172,90,Running
458,339096552,B-Wolf,163,87,128.735990,86,Bicycling
471,954337505,B-Wolf,139,82,124.632659,86,Running


There's more than one way to do this. The code block below will give the same results as the one above. Spend a minute looking at it until you can tell why. It may help to think about both filters in chunks in pieces (both filters are two sets of two linked expressions).

In [15]:
df[~((df['Heart Rate Avg'] < 120) | (df['Heart Rate Avg'] > 150)) &
   ~((df['Heart Rate Avg'] > 130) & (df['Heart Rate Avg'] < 140))]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
4,157561470,Skykandal,157,76,146.046041,47,Swimming
8,198499198,Skykandal,145,101,126.178451,30,Running
14,760138653,B-Wolf,140,82,121.886573,22,Running
29,616103286,Skykandal,156,74,126.509807,82,Bicycling
36,512580058,B-Wolf,201,86,147.932365,28,Running
...,...,...,...,...,...,...,...
448,924174572,Skykandal,190,60,142.296311,69,Swimming
449,508684249,Skykandal,151,82,141.705172,90,Running
458,339096552,B-Wolf,163,87,128.735990,86,Bicycling
471,954337505,B-Wolf,139,82,124.632659,86,Running


Remember how I said that making intermediate data frames is bad practice? When the expression gets complicated enough, you sometimes need to perform intermediate steps for debugging purposes. However, you can potentially build extremely powerful filters this way.

#### Below, write code that filters for people who went running or swimming while using the B-Wolf device OR who went bicycling using Starkandal, and have an average heart rate below the number you get by averaging their maximum and minimum heart rates together.

In [ ]:
# your code goes here


At this point you may be thinking, "Wow, I wish I could write a function, pass all of these rows to that function, evaluate the row there, and then just return a True/False label I could filter against."

That's a great idea! We'll cover it in the next notebook.